In [1]:
import os
import tensorflow as tf
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import models
from tensorflow.keras import layers
from sklearn.model_selection import KFold
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils as np_utils
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import CSVLogger

In [2]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [ ]:
# get the features from the respective directories and label them to create dataset
orig_dir=os.getcwd()
orig_dir

In [26]:
# Retrieve Full HOF Data
features_data_hof_full=[]
features_label_hof_full=[]

path="F:\CA2b\HOF_Full_Classified"
folders=os.listdir(path)
t_count=0
for f in folders:
    count=1
    folder_path=os.path.join(path, f)
    files=os.listdir(folder_path)
    for file in files:
        file_path=os.path.join(folder_path,file)
        features=os.listdir(file_path)
        for feature in features:
            feature_path=os.path.join(file_path,feature)
            histogram=np.load(feature_path)
            histogram = np.mean(histogram, axis=0)
            features_data_hof_full.append(histogram)
            features_label_hof_full.append(f)
            count=count+1
            t_count=t_count+1

In [27]:
# Process HOF Type to correct format for DNN
features_data_hof_full=np.reshape(features_data_hof_full, (167796, 255)).T
print(features_data_hof_full.shape)


(255, 167796)


In [15]:
# Retrieve Full LBP Data
features_data_l=[]
features_label_l=[]

path="F:\CA2b\LBP_Top_Raw_Classified"
folders=os.listdir(path)
t_count=0
for f in folders:
    count=1
    folder_path=os.path.join(path, f)
    
    files=os.listdir(folder_path)
    for file in files:
        file_path=os.path.join(folder_path,file)
        
        features=os.listdir(file_path)
        for feature in features:
            feature_path=os.path.join(file_path,feature)
            histogram=np.load(feature_path)
            histogram = np.mean(histogram, axis=0)
            features_data_l.append(histogram)
            features_label_l.append(f)
            count=count+1
            t_count=t_count+1

In [17]:
# Process to correct format for DNN
features_data_l=np.concatenate(features_data_l, axis=1).T
features_data_l
print(features_data_l.shape)


(255, 768)


In [ ]:
# Retrieve LBP_Eye data
features_data_le=[]
features_label_le=[]

path="F:\CA2b\Eye_features_Classified_1"
folders=os.listdir(path)
t_count=0
for f in folders:
    count=1
    folder_path=os.path.join(path, f)
    files=os.listdir(folder_path)
    for file in files:
        file_path=os.path.join(folder_path,file)
        features=os.listdir(file_path)
        for feature in features:
            feature_path=os.path.join(file_path,feature)
            histogram=np.load(feature_path)
            histogram = np.mean(histogram, axis=0)
            features_data_le.append(histogram)
            features_label_le.append(f)
            count=count+1
            t_count=t_count+1

In [ ]:
# Process to correct format for DNN
features_data_le=np.concatenate(features_data_le, axis=1).T
print(features_data_le.shape)


In [ ]:
# Retrieve LBP_Mouth data
features_data_lm=[]
features_label_lm=[]
path=r'F:\CA2b\Nose_Features_classified_1'
folders=os.listdir(path)
t_count=0
for f in folders:
    count=1
    folder_path=os.path.join(path, f)
    files=os.listdir(folder_path)
    for file in files:
        file_path=os.path.join(folder_path,file)
        features=os.listdir(file_path)
        for feature in features:
            feature_path=os.path.join(file_path,feature)
            histogram=np.load(feature_path)
            histogram = np.mean(histogram, axis=0)
            features_data_lm.append(histogram)
            features_label_lm.append(f)
            count=count+1
            t_count=t_count+1

In [ ]:
# Process to correct format for DNN
features_data_lm=np.concatenate(features_data_lm, axis=1).T
print(features_data_lm.shape)

In [3]:
# Retrieve HOF data for eye and mouth and cropped
cropped_hof_path='F:\CA2b\HOF_combined_dataset\hof_feature_128x64, feature_hof.npy'
eye_nose_hof_path='F:\CA2b\HOF_combined_dataset\hof_feature_eye_nose, feature_hof.npy'
other_label_path='F:\CA2b\HOF_combined_dataset\label_hof_fullimage, feature_hof.npy'
features_data_hof_eye_mouth=np.load(eye_nose_hof_path)
features_data_hof_cropped=np.load(cropped_hof_path)
label_other=np.load(other_label_path)


In [45]:
# combine two types of features if needed
X_Combined=np.concatenate((features_data_hof_cropped,features_data_hof_eye_mouth),axis=1)
X_Combined.shape

(255, 4428)

X=features_data
y=features_label

In [46]:
#X=features_data
X=X_Combined
y=label_other
X.shape

(255, 4428)

In [40]:
# One hot encoder of labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
dummy_y = np_utils.to_categorical(integer_encoded)

In [48]:
save_location='F:\CA2b\Run_time_data'
modelname= 'RB_FF_NN13'
savelocation=os.path.join(save_location, modelname)
filepath=savelocation + ".hdf5"
csv_logger=CSVLogger(savelocation +'.csv')
#callbacks_list  = [checkpoint,csv_logger,LRScheduler]

In [49]:

model = Sequential()
model.add(Dense(512, activation='sigmoid', input_shape=(4428,)))
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(7, activation='softmax'))
model.summary()
          
          

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 512)               2267648   
_________________________________________________________________
dense_31 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_32 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_33 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_34 (Dense)             (None, 7)                 3591      
Total params: 3,059,207
Trainable params: 3,059,207
Non-trainable params: 0
_________________________________________________________________


In [50]:
callbacks=[csv_logger, ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True)]


In [51]:
opt='RMSProp'
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size = 0.20)
history = model.fit(X_train, y_train, batch_size=2, epochs=200, verbose=1,validation_split=0.005, callbacks=callbacks)

Train on 202 samples, validate on 2 samples
Epoch 1/200
202/202 [==============================] - 10s 51ms/sample - loss: 1.8020 - accuracy: 0.3069 - val_loss: 1.4028 - val_accuracy: 0.0000e+00
Epoch 2/200
202/202 [==============================] - 9s 44ms/sample - loss: 1.6862 - accuracy: 0.3366 - val_loss: 1.6794 - val_accuracy: 0.0000e+00
Epoch 3/200
202/202 [==============================] - 10s 49ms/sample - loss: 1.6755 - accuracy: 0.2871 - val_loss: 1.0818 - val_accuracy: 1.0000
Epoch 4/200
202/202 [==============================] - 9s 46ms/sample - loss: 1.6728 - accuracy: 0.3168 - val_loss: 0.7145 - val_accuracy: 1.0000
Epoch 5/200
202/202 [==============================] - 10s 48ms/sample - loss: 1.6817 - accuracy: 0.3416 - val_loss: 0.7520 - val_accuracy: 1.0000
Epoch 6/200
202/202 [==============================] - 10s 48ms/sample - loss: 1.6435 - accuracy: 0.3515 - val_loss: 0.6103 - val_accuracy: 1.0000
Epoch 7/200
202/202 [==============================] - 9s 46ms/sampl

202/202 [==============================] - 10s 48ms/sample - loss: 0.4201 - accuracy: 0.8713 - val_loss: 0.0879 - val_accuracy: 1.0000
Epoch 112/200
202/202 [==============================] - 9s 46ms/sample - loss: 0.5778 - accuracy: 0.8416 - val_loss: 0.0122 - val_accuracy: 1.0000
Epoch 113/200
202/202 [==============================] - 9s 47ms/sample - loss: 0.4384 - accuracy: 0.8812 - val_loss: 0.0592 - val_accuracy: 1.0000
Epoch 114/200
202/202 [==============================] - 10s 48ms/sample - loss: 0.5107 - accuracy: 0.8465 - val_loss: 0.9417 - val_accuracy: 0.5000
Epoch 115/200
202/202 [==============================] - 10s 48ms/sample - loss: 0.6313 - accuracy: 0.8812 - val_loss: 0.0126 - val_accuracy: 1.0000
Epoch 116/200
202/202 [==============================] - 10s 48ms/sample - loss: 0.3769 - accuracy: 0.8812 - val_loss: 0.0661 - val_accuracy: 1.0000
Epoch 117/200
202/202 [==============================] - 9s 47ms/sample - loss: 0.4209 - accuracy: 0.8812 - val_loss: 0.01

202/202 [==============================] - 9s 47ms/sample - loss: 0.4093 - accuracy: 0.8663 - val_loss: 0.1609 - val_accuracy: 1.0000
Epoch 167/200
202/202 [==============================] - 9s 45ms/sample - loss: 0.5670 - accuracy: 0.9059 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 168/200
202/202 [==============================] - 10s 47ms/sample - loss: 0.2967 - accuracy: 0.9158 - val_loss: 3.1447e-04 - val_accuracy: 1.0000
Epoch 169/200
202/202 [==============================] - 10s 48ms/sample - loss: 0.2637 - accuracy: 0.9158 - val_loss: 2.4469e-04 - val_accuracy: 1.0000
Epoch 170/200
202/202 [==============================] - 9s 46ms/sample - loss: 0.2437 - accuracy: 0.9505 - val_loss: 4.6311e-05 - val_accuracy: 1.0000
Epoch 171/200
202/202 [==============================] - 9s 46ms/sample - loss: 0.7695 - accuracy: 0.8812 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 172/200
202/202 [==============================] - 9s 47ms/sample - loss: 0.3125 - accuracy: 0.9158 - val_

In [52]:

yhat_classes = model.predict_classes(X_test, verbose=0)
y_test_original = y_test.argmax(1)
y_train_original = y_train.argmax(1)
labels = np.unique(y_test_original)
a= confusion_matrix(y_test_original, yhat_classes)
print(a)
print(classification_report(y_test_original, yhat_classes))


[[10  0  0  0  0  0  1]
 [ 1  0  0  0  0  0  0]
 [ 2  0  8  0  1  0  0]
 [ 0  0  0 17  1  0  2]
 [ 0  0  0  0  3  0  0]
 [ 0  0  0  0  0  1  1]
 [ 0  0  0  0  1  0  2]]
              precision    recall  f1-score   support

           0       0.77      0.91      0.83        11
           1       0.00      0.00      0.00         1
           2       1.00      0.73      0.84        11
           3       1.00      0.85      0.92        20
           4       0.50      1.00      0.67         3
           5       1.00      0.50      0.67         2
           6       0.33      0.67      0.44         3

    accuracy                           0.80        51
   macro avg       0.66      0.66      0.62        51
weighted avg       0.86      0.80      0.81        51



C:\Users\User\Anaconda3\envs\classifier6\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
